In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import json


In [10]:
experiment_folder = "../data/experiment-i"
subject_info_csv = "../data/subject_info.csv"  # Ensure this file contains height & weight data


df_subjects = pd.read_csv(subject_info_csv)  # Expected: Subject-Number, Height, Weight
df_subjects["Subject-Number"] = df_subjects["Subject-Number"].apply(lambda x: f"S{x}")  # Convert to "S1", "S2", etc.

subject_data = []

subject_folders = sorted(
    [f for f in glob.glob(os.path.join(experiment_folder, "S*")) if os.path.isdir(f)]
)

for subject_path in subject_folders:
    subject_id = os.path.basename(subject_path)  # Extract "S1", "S2", etc.

    # Retrieve height & weight for the subject
    subject_info = df_subjects[df_subjects["Subject-Number"] == subject_id]

    height = int(subject_info["Height/cm"].values[0])
    weight = int(subject_info["Weight/kg"].values[0])

    subject_entry = {
        "subject": subject_id,
        "height": height,
        "weight": weight
    }

    for posture_id in ["1", "2", "3"]:
        txt_file = os.path.join(subject_path, f"{posture_id}.txt")

        # Read and process file
        frames = []
        with open(txt_file, "r") as f:
            for line in f:
                line = line.strip()
                if line and not any(c.isalpha() for c in line):  # Ignore headers, non-numeric lines
                    frames.append([float(x) for x in line.split("\t")])  # Convert row into a list of floats

        # Convert to NumPy array and apply pressure conversion (0-1000 → 0-100 mmHg)
        frames_array = np.array(frames) * 0.1  # Convert pressure values to mmHg
        num_frames = frames_array.shape[0]

        if num_frames > 2:  # Ensure at least 3 frames exist
            frames_array = frames_array[2:]  # Start from frame 3
            avg_frame = np.mean(frames_array, axis=0)  # Compute average frame

            avg_frame_reshaped = avg_frame.reshape(64, 32).tolist()

            # Store average frame in subject dictionary
            subject_entry[f"posture{posture_id}"] = avg_frame_reshaped

    # Append subject entry to final JSON structure
    subject_data.append(subject_entry)


output_json_file = "../data/experiment1_data.json"
with open(output_json_file, "w") as json_file:
    json.dump(subject_data, json_file, indent=4)
